In [ ]:
# Import public packages and functions
import os
import pandas as pd
import numpy as np
import pickle
import sys
import json
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt

# inserting the lib folder to the compiler
sys.path.insert(0, './lib')
sys.path.insert(0, './utils/')

import warnings
warnings.filterwarnings("ignore")

from lib_event import EVENTS
from lib_accelerometer import ACCELEROMETER
from lib_data import DATA_IO

import utils_plotting_tapping, utils_misc, utils_accelerometer

In [ ]:
PATH_CURR   = os.path.abspath(os.curdir)    # current code
PATH        = (str(Path(PATH_CURR).parent)) # data repository: upper directory where datasets situated
PATH_DATA   = PATH + "\\data"               # the recordings data directory
PATH_EVENTS = PATH + "\\events"
SUB_LIST    = utils_misc.get_SUB_list(PATH_DATA) # get the SUB id list which we have a recording of them

# 1. ACCELEROMETER EVENTS

## 1.1 Plot All Tapping Periods  Across Patients

In [ ]:
for SUB in SUB_LIST:
    
    SUB_ACCELEROMETER = ACCELEROMETER(PATH, SUB)
    SUB_EVENTS        = EVENTS(PATH, SUB, 'acc_right')

    tapping_period_indices = utils_accelerometer.get_task_period_times_indices(SUB_EVENTS.period_tap)
    tapping_R_indices      = utils_accelerometer.get_task_period_times_indices(SUB_EVENTS.right_tap)
    tapping_L_indices      = utils_accelerometer.get_task_period_times_indices(SUB_EVENTS.left_tap)

    for i in range(len(tapping_period_indices)):
        utils_plotting_tapping.plot_tapping_period(SUB_ACCELEROMETER, tapping_period_indices, tapping_period_no=i)

## 1.2 Observe Tapping Behavior for Tapping Task Periods for Every Patient

In [ ]:
df_tapping_count = pd.DataFrame(columns=["patient", "tap_period", "hand", "count"])

for SUB in SUB_LIST:
    SUB_ACCELEROMETER      = ACCELEROMETER(PATH, SUB)
    SUB_EVENTS             = EVENTS(PATH, SUB, 'acc_right')
    
    tapping_period_indices = utils_accelerometer.get_task_period_times_indices(SUB_EVENTS.period_tap)
    tapping_R_indices      = utils_accelerometer.get_task_period_times_indices(SUB_EVENTS.right_tap)
    tapping_L_indices      = utils_accelerometer.get_task_period_times_indices(SUB_EVENTS.left_tap)

    for tapping_period_id in range(len(tapping_period_indices)):
        
        tapping_task_start_index  = tapping_period_indices[tapping_period_id][0]
        tapping_task_finish_index = tapping_period_indices[tapping_period_id][1]
        
        right_tap_indices         = [t for t in tapping_R_indices if tapping_task_start_index < t[0] < tapping_task_finish_index]
        left_tap_indices          = [t for t in tapping_L_indices if tapping_task_start_index < t[0] < tapping_task_finish_index]

        row_left                  = {}
        row_left["patient"]       = SUB
        row_left["tap_period"]    = tapping_period_id + 1
        row_left["hand"]          = "left"
        row_left["count"]         = len(left_tap_indices)

        row_right                 = {}
        row_right["patient"]      = SUB
        row_right["tap_period"]   = tapping_period_id + 1
        row_right["hand"]         = "right"
        row_right["count"]        = len(right_tap_indices)

        df_tapping_count.loc[len(df_tapping_count)] = row_right 
        df_tapping_count.loc[len(df_tapping_count)] = row_left

## 1.3 Show Accelerometer Data

### methods

In [ ]:
def plot_aligned_acceleremetor_recordings(tapping_period_indices, tapping_R_indices, tapping_L_indices, fs, plot_type, alignment):

    for tapping_period_id in range(len(tapping_period_indices)):

        # get the start and finish indices of selected tapping period of the patient
        tapping_task_start_index  = tapping_period_indices[tapping_period_id][0]
        tapping_task_finish_index = tapping_period_indices[tapping_period_id][1]
    
        # get the start and finish indices of both right and left hand taps during this tapping period
        right_tap_indices         = [t for t in tapping_R_indices if tapping_task_start_index < t[0] < tapping_task_finish_index]
        left_tap_indices          = [t for t in tapping_L_indices if tapping_task_start_index < t[0] < tapping_task_finish_index]
    
        # to store the raw recordings from accelerometer data
        right_taps = []
        left_taps  = []
    
        if(plot_type=="raw"):
            for right_i in range(len(right_tap_indices)):
                if(alignment=="onset"):
                    tap_start_index  = right_tap_indices[right_i][0] - 2*fs
                    tap_finish_index = tap_start_index + 4*fs
                    tap              = SUB_ACCELEROMETER.ACC_R[tap_start_index:tap_finish_index]
                else:
                    tap_start_index  = right_tap_indices[right_i][1] - 2*fs
                    tap_finish_index = tap_start_index + 4*fs
                    tap              = SUB_ACCELEROMETER.ACC_R[tap_start_index:tap_finish_index]
                right_taps.append(tap)
            
            for left_i in range(len(left_tap_indices)):
                if(alignment=="onset"):
                    tap_start_index  = left_tap_indices[left_i][0] - 2*fs
                    tap_finish_index = tap_start_index + 4*fs
                    tap              = SUB_ACCELEROMETER.ACC_L[tap_start_index:tap_finish_index]
                else:
                    tap_start_index  = left_tap_indices[left_i][1] - 2*fs
                    tap_finish_index = tap_start_index + 4*fs
                    tap              = SUB_ACCELEROMETER.ACC_L[tap_start_index:tap_finish_index]
                left_taps.append(tap)
                
        elif(plot_type=="nan-filled"):
            for right_i in range(len(right_tap_indices)):
                if(alignment=="onset"):
                    tap_start_index  = right_tap_indices[right_i][0] - 2*fs
                    tap_finish_index = right_tap_indices[right_i][1]
                    tap              = SUB_ACCELEROMETER.ACC_R[tap_start_index:tap_finish_index]
                    if(len(tap)<fs*4):
                        tap = np.pad(tap, (0, fs*4- len(tap)), constant_values=np.nan)
                    else:
                        tap = tap[0:fs*4]
                    right_taps.append(tap)
                else:
                    tap_start_index  = right_tap_indices[right_i][1] - (right_tap_indices[right_i][1]-right_tap_indices[right_i][0])
                    tap_finish_index = right_tap_indices[right_i][1] + 2*fs
                    tap              = SUB_ACCELEROMETER.ACC_R[tap_start_index:tap_finish_index]
                    if(len(tap)<fs*4):
                        tap = np.pad(tap, (fs*4- len(tap),0), constant_values=np.nan) # pad the front
                    else:
                        tap = tap[-fs*4:]
                    right_taps.append(tap)      

            for left_i in range(len(left_tap_indices)):
                if(alignment=="onset"):
                    tap_start_index  = left_tap_indices[left_i][0] - 2*fs
                    tap_finish_index = left_tap_indices[left_i][1]
                    tap              = SUB_ACCELEROMETER.ACC_L[tap_start_index:tap_finish_index]
                    if(len(tap)<fs*4):
                        tap = np.pad(tap, (0, fs*4- len(tap)), constant_values=np.nan)
                    else:
                        tap = tap[0:fs*4]
                    left_taps.append(tap)
                else:
                    tap_start_index  = left_tap_indices[left_i][1] - (left_tap_indices[left_i][1]-left_tap_indices[left_i][0])
                    tap_finish_index = left_tap_indices[left_i][1] + 2*fs
                    tap              = SUB_ACCELEROMETER.ACC_L[tap_start_index:tap_finish_index]
                    if(len(tap)<fs*4):
                        tap = np.pad(tap, (fs*4- len(tap),0), constant_values=np.nan) # pad the front
                    else:
                        tap = tap[-fs*4:]
                    left_taps.append(tap)

        # plot tap duration
        duration_R_taps = [(t[1]-t[0]) / fs for t in right_tap_indices]
        duration_L_taps = [(t[1]-t[0]) / fs for t in left_tap_indices]
        
        boxprops = plt.boxplot([duration_L_taps, duration_R_taps], labels=['left hand', 'right hand'], vert=False, patch_artist=True,
                        medianprops= dict(color='white', linewidth=5))
        for patch, color in zip(boxprops['boxes'], ['blue', 'red']):
            patch.set_facecolor(color)
        plt.title("Tapping Period No: " + str(tapping_period_id+1) + " - Tap Durations")
        plt.xlabel("time [seconds]")
        plt.show()

        # plot inter-tap intervals
        start_r_indices     = np.array([t[0] for t in right_tap_indices])
        tapping_r_distances = (start_r_indices[1:] - start_r_indices[0:-1]) / fs
        start_l_indices     = np.array([t[0] for t in left_tap_indices])
        tapping_l_distances = (start_l_indices[1:] - start_l_indices[0:-1]) / fs
        
        plt.hist(tapping_r_distances, bins= np.linspace(0, 30, 29), alpha = 0.5, label='right mean', color="r")
        plt.hist(tapping_l_distances, bins= np.linspace(0, 30, 29), alpha = 0.5, label='left mean', color="b")
        plt.legend(loc="upper right")
        plt.xlabel("time [seconds]")
        plt.title("Tapping Period No: " + str(tapping_period_id+1) + " - Inter Tapping Interval")
        plt.xlim([0,29])
        plt.show()

        # Plot aligned taps
        t           = np.linspace(-2, 2, fs*4) 
        mean_right  = np.nanmean(right_taps, axis=0)
        error_right = 2 * np.std(right_taps, axis=0) / np.sqrt(len(right_taps))
        mean_left   = np.nanmean(left_taps, axis=0)
        error_left  = 2 * np.std(left_taps, axis=0) / np.sqrt(len(left_taps))

        try:
            plt.plot(t, mean_right, label=' right mean', c="r", linewidth=1)
            plt.fill_between(t, mean_right - error_right, mean_right + error_right, alpha=0.2, color="r")
        except:
            pass
            
        try:
            plt.plot(t, mean_left, label='left mean', c="b", linewidth=1)
            plt.fill_between(t, mean_left - error_left, mean_left + error_left, alpha=0.2, color="b")
        except:
            pass
        plt.xlabel("time [seconds]")
        plt.title("Tapping Period No: " + str(tapping_period_id+1) + " ["+ alignment +" aligned]")
        plt.axvline(x=0, color='k', label=alignment)
        plt.legend(loc="upper left")
        plt.xlim([-2,2])
        plt.ylim([0,2e-6])
        plt.show()

def filter_events_by_time_threshold(event_indices, fs, time_threshold):
    # Convert minimum time from seconds to samples
    min_time_samples = fs * time_threshold
    filtered_events  = [event_indices[0]]  # Start with the first event
    deleted_events   = []  # List to store deleted events

    # Iterate over the remaining events
    for i in range(1, len(event_indices)):
        start_current, end_current = event_indices[i]
        _, end_previous = filtered_events[-1]
        
        # Check if the current event starts at least min_time_samples after the previous event
        if((start_current - end_previous) >= min_time_samples):
            filtered_events.append(event_indices[i])
        else:
            deleted_events.append(event_indices[i])

    return filtered_events, deleted_events

def remove_long_taps(event_indices, fs, time_threshold):
    # Convert minimum time from seconds to samples
    max_tap_length  = fs * time_threshold
    filtered_events = []
    for i in range(1, len(event_indices)):
        start_i, end_i = event_indices[i]
        if((end_i - start_i) <= max_tap_length):
            filtered_events.append(event_indices[i])

    return filtered_events

def tap_counts_in_tapping_periods(tapping_period_indices, tapping_R_indices, tapping_L_indices):

    for tapping_period_id in range(len(tapping_period_indices)):

        # get the start and finish indices of selected tapping period of the patient
        tapping_task_start_index  = tapping_period_indices[tapping_period_id][0]
        tapping_task_finish_index = tapping_period_indices[tapping_period_id][1]
    
        # get the start and finish indices of both right and left hand taps during this tapping period
        right_tap_indices         = [t for t in tapping_R_indices if tapping_task_start_index < t[0] < tapping_task_finish_index]
        left_tap_indices          = [t for t in tapping_L_indices if tapping_task_start_index < t[0] < tapping_task_finish_index]
        
        print("Tapping Period: " + str(tapping_period_id + 1))
        print("---> right : " + str(len(right_tap_indices)))
        print("---> left  : " + str(len(left_tap_indices)))

def plot_single_tap(tap_indices, index, fs, hand, alignment):
    if(alignment=="onset"):
        tap_start_index  = tap_indices[index][0] - 2*fs
        tap_finish_index = tap_indices[index][1]
        
        if(hand=="right"):
            tap = SUB_ACCELEROMETER.ACC_R[tap_start_index:tap_finish_index]
        else:
            tap = SUB_ACCELEROMETER.ACC_L[tap_start_index:tap_finish_index]
            
        if(len(tap)<fs*4):
            tap = np.pad(tap, (0, fs*4- len(tap)), constant_values=np.nan)
        else:
            tap = tap[0:fs*4]
    else:
        tap_start_index  = tap_indices[index][1] - (tap_indices[index][1]-tap_indices[index][0])
        tap_finish_index = tap_indices[index][1] + 2*fs
        if(hand=="right"):
            tap = SUB_ACCELEROMETER.ACC_R[tap_start_index:tap_finish_index]
        else:
            tap = SUB_ACCELEROMETER.ACC_L[tap_start_index:tap_finish_index]
        if(len(tap)<fs*4):
            tap = np.pad(tap, (fs*4- len(tap),0), constant_values=np.nan) # pad the front
        else:
            tap = tap[-fs*4:] 

    t = np.linspace(-2, 2, fs*4)
    if(hand=="right"):
        plt.plot(t, tap, c="r", linewidth=1)
    else:
        plt.plot(t, tap, c="b", linewidth=1)
    plt.axvline(x=0, color='k', label=alignment)
    plt.ylim([0,2e-6])
    plt.title(hand + " hand:  index " + str(index))
    plt.show()

def get_tapping_count_df(tapping_period_indices, tapping_R_indices, tapping_L_indices):
    
    df_tapping_count = pd.DataFrame(columns=["tap_period", "hand", "count"])
    
    for tapping_period_id in range(len(tapping_period_indices)):
        
        tapping_task_start_index  = tapping_period_indices[tapping_period_id][0]
        tapping_task_finish_index = tapping_period_indices[tapping_period_id][1]
        
        right_tap_indices         = [t for t in tapping_R_indices if tapping_task_start_index < t[0] < tapping_task_finish_index]
        left_tap_indices          = [t for t in tapping_L_indices if tapping_task_start_index < t[0] < tapping_task_finish_index]

        row_left                  = {}
        row_left["tap_period"]    = tapping_period_id + 1
        row_left["hand"]          = "left"
        row_left["count"]         = len(left_tap_indices)

        row_right                 = {}
        row_right["tap_period"]   = tapping_period_id + 1
        row_right["hand"]         = "right"
        row_right["count"]        = len(right_tap_indices)

        df_tapping_count.loc[len(df_tapping_count)] = row_right 
        df_tapping_count.loc[len(df_tapping_count)] = row_left
        
    return df_tapping_count

def add_counts(ax):
    for p in ax.patches:
        ax.text(p.get_x() + p.get_width() / 2,   # X position
                p.get_height(),                  # Y position
                f'{int(p.get_height())}',        # Text (count)
                ha='center', va='bottom')        # Center-align the text

def plot_tapping_count_by_tap_periods(df_tapping_count):
    ax = sns.barplot(data=df_tapping_count, x="tap_period", y="count", hue="hand", palette=["r","b"])
    add_counts(ax)
    plt.ylim([0,50])

def select_task_tappings(tapping_period_indices, tapping_R_indices, tapping_L_indices):
    
    right_indices = [] 
    left_indices  = []
    
    for tapping_period_id in range(len(tapping_period_indices)):

        # get the start and finish indices of selected tapping period of the patient
        tapping_task_start_index  = tapping_period_indices[tapping_period_id][0]
        tapping_task_finish_index = tapping_period_indices[tapping_period_id][1]
    
        # get the start and finish indices of both right and left hand taps during this tapping period
        right_indices.extend([t for t in tapping_R_indices if tapping_task_start_index < t[0] < tapping_task_finish_index])
        left_indices.extend([t for t in tapping_L_indices if tapping_task_start_index < t[0] < tapping_task_finish_index])

    return right_indices, left_indices

from scipy.signal import find_peaks

def custom_tap_finding(SUB_ACCELEROMETER, hand, tap_period_start,tap_period_end):

    fs = int(SUB_ACCELEROMETER.fs)  # Sampling frequency

    if(hand=="right"):
        svm = SUB_ACCELEROMETER.ACC_R[tap_period_start:tap_period_end]
    else:
        svm = SUB_ACCELEROMETER.ACC_L[tap_period_start:tap_period_end]
        
    # Find peaks in the SVM (Signal Vector Magnitude)
    x_peaks, _ = find_peaks(svm, height=1e-6, distance=3*fs)

    # Find indices of tap start and end
    peak_indices = []
    margin = int(fs / 4)
    
    for i in x_peaks:
        i_start, i_end = False, False
        temp_i = i
        
        # Find start: first value below threshold before peak
        while not i_start:
            temp_i -= 1
            if all(svm[temp_i-margin:temp_i] < 0.1e-6):
                i_start = temp_i
        
        # Find end: first value below threshold after peak
        temp_i = i
        while not i_end:
            temp_i += 1
            if all(svm[temp_i:temp_i+margin] < 0.1e-6):
                i_end = temp_i
        
        # Add start, peak, and end indices to the list
        peak_indices.append([i, i_start, i_end])
    
    # Convert the list to a numpy array (n-taps x [peak, start, end])
    peak_indices = np.array(peak_indices)
    peak_indices = [(i[1]+tap_period_start,i[2]+tap_period_start) for i in peak_indices]

    return peak_indices

### operations

In [ ]:
SUB                    = "019"
SUB_ACCELEROMETER      = ACCELEROMETER(PATH, SUB)
SUB_EVENTS             = EVENTS(PATH, SUB, 'acc_right')
    
tapping_period_indices = utils_accelerometer.get_task_period_times_indices(SUB_EVENTS.period_tap)
tapping_R_indices      = utils_accelerometer.get_task_period_times_indices(SUB_EVENTS.right_tap)
tapping_L_indices      = utils_accelerometer.get_task_period_times_indices(SUB_EVENTS.left_tap)

if((SUB=="010") or (SUB=="105")) :
    
    if(SUB=="010"):
        del tapping_period_indices[0]
        del tapping_period_indices[1]
        
    tapping_R_indices_filtered = []
    tapping_L_indices_filtered = []

    for i in range(len(tapping_period_indices)):
        tap_period_start           = tapping_period_indices[i][0]
        tap_period_end             = tapping_period_indices[i][1]
        tapping_R_indices_filtered.extend(custom_tap_finding(SUB_ACCELEROMETER, "right", tap_period_start, tap_period_end))
        tapping_L_indices_filtered.extend(custom_tap_finding(SUB_ACCELEROMETER, "left", tap_period_start, tap_period_end))

    tapping_R_indices_filtered, tapping_R_indices_removed  = filter_events_by_time_threshold(tapping_R_indices_filtered, fs=512, time_threshold=4)
    tapping_L_indices_filtered, tapping_L_indices_removed  = filter_events_by_time_threshold(tapping_L_indices_filtered, fs=512, time_threshold=4)
    
else:
    
    tapping_R_indices_filtered, tapping_L_indices_filtered = select_task_tappings(tapping_period_indices, tapping_R_indices, tapping_L_indices) 
    
    tapping_R_indices_filtered, tapping_R_indices_removed  = filter_events_by_time_threshold(tapping_R_indices_filtered, fs=512, time_threshold=4)
    tapping_L_indices_filtered, tapping_L_indices_removed  = filter_events_by_time_threshold(tapping_L_indices_filtered, fs=512, time_threshold=4)
    
    print("right hemisphere removed overlapping taps: " + str(len(tapping_R_indices_removed)))
    print("left hemisphere removed overlapping taps: " + str(len(tapping_L_indices_removed)))

#### plot single taps

In [ ]:
for index in range(len(tapping_R_indices_filtered)):
    plot_single_tap(tapping_R_indices_filtered, index=index, fs=512, hand="right", alignment="onset")
    plot_single_tap(tapping_R_indices_filtered, index=index, fs=512, hand="right", alignment="offset")
    print("--------------------------------------------------------------------------")
    print("--------------------------------------------------------------------------")
    print("--------------------------------------------------------------------------")
    
for index in range(len(tapping_L_indices_filtered)):
    plot_single_tap(tapping_L_indices_filtered, index=index, fs=512, hand="left", alignment="onset")
    plot_single_tap(tapping_L_indices_filtered, index=index, fs=512, hand="left", alignment="offset")
    print("--------------------------------------------------------------------------")
    print("--------------------------------------------------------------------------")
    print("--------------------------------------------------------------------------")

#### update taps borders

In [ ]:
fs = 512
#tapping_L_indices_filtered[34] = (int(tapping_L_indices_filtered[34][0]),tapping_L_indices_filtered[34][1]-1*fs)
#tapping_R_indices_filtered[33] = (int(tapping_R_indices_filtered[33][0]+1*fs),int(tapping_R_indices_filtered[33][1]-3.5*fs))

In [ ]:
tapping_R_indices_filtered[28] = (int(tapping_R_indices_filtered[28][0]),int(tapping_R_indices_filtered[28][1]+0.2*fs))
tapping_L_indices_filtered[15] = (int(tapping_L_indices_filtered[15][0]+1.2*fs),int(tapping_L_indices_filtered[15][1]-0.5*fs))
tapping_L_indices_filtered[18] = (int(tapping_L_indices_filtered[18][0]+1.65*fs),int(tapping_L_indices_filtered[18][1]))
tapping_L_indices_filtered[28] = (int(tapping_L_indices_filtered[28][0]),int(tapping_L_indices_filtered[28][1]-1*fs))
tapping_L_indices_filtered[30] = (int(tapping_L_indices_filtered[30][0]),int(tapping_L_indices_filtered[30][1]-0.8*fs))
tapping_L_indices_filtered[34] = (int(tapping_L_indices_filtered[34][0]),int(tapping_L_indices_filtered[34][1]-2*fs))

In [ ]:
fs    = 512
index = 34
hand  = "left"

if(hand=="right"):
    print("length : " + str((tapping_R_indices_filtered[index][1]-tapping_R_indices_filtered[index][0]) / fs) + " seconds")
else:
    print("length : " + str((tapping_L_indices_filtered[index][1]-tapping_L_indices_filtered[index][0]) / fs) + " seconds")
    
if(hand=="right"):
    plot_single_tap([(int(tapping_R_indices_filtered[index][0]),tapping_R_indices_filtered[index][1])], index=0, fs=512, hand=hand, alignment="onset")
    plot_single_tap([(int(tapping_R_indices_filtered[index][0]),tapping_R_indices_filtered[index][1])], index=0, fs=512, hand=hand, alignment="offset")
else:
    plot_single_tap([(int(tapping_L_indices_filtered[index][0]),tapping_L_indices_filtered[index][1])], index=0, fs=512, hand=hand, alignment="onset")
    plot_single_tap([(int(tapping_L_indices_filtered[index][0]),tapping_L_indices_filtered[index][1])], index=0, fs=512, hand=hand, alignment="offset")

In [ ]:
# correct onset
correction = 0.5

if(hand=="right"):
    plot_single_tap([(int(tapping_R_indices_filtered[index][0]+correction*fs),tapping_R_indices_filtered[index][1])], index=0, fs=512, hand=hand, alignment="onset")
    plot_single_tap([(int(tapping_R_indices_filtered[index][0]+correction*fs),tapping_R_indices_filtered[index][1])], index=0, fs=512, hand=hand, alignment="offset")
else:
    plot_single_tap([(int(tapping_L_indices_filtered[index][0]+correction*fs),tapping_L_indices_filtered[index][1])], index=0, fs=512, hand=hand, alignment="onset")
    plot_single_tap([(int(tapping_L_indices_filtered[index][0]+correction*fs),tapping_L_indices_filtered[index][1])], index=0, fs=512, hand=hand, alignment="offset")

In [ ]:
# correct offset

correction = -2

if(hand=="right"):
    plot_single_tap([(int(tapping_R_indices_filtered[index][0]),int(tapping_R_indices_filtered[index][1]+correction*fs))], index=0, fs=512, hand=hand, alignment="onset")
    plot_single_tap([(int(tapping_R_indices_filtered[index][0]),int(tapping_R_indices_filtered[index][1]+correction*fs))], index=0, fs=512, hand=hand, alignment="offset")
else:
    plot_single_tap([(int(tapping_L_indices_filtered[index][0]),int(tapping_L_indices_filtered[index][1]+correction*fs))], index=0, fs=512, hand=hand, alignment="onset")
    plot_single_tap([(int(tapping_L_indices_filtered[index][0]),int(tapping_L_indices_filtered[index][1]+correction*fs))], index=0, fs=512, hand=hand, alignment="offset")

#### remove uncorrect taps

In [ ]:
remove_R_indices = [2,3,4,5,32]
remove_L_indices = [64,65,103]
tapping_R_indices_corrected = [value for i, value in enumerate(tapping_R_indices_filtered) if i not in remove_R_indices]
tapping_L_indices_corrected = [value for i, value in enumerate(tapping_L_indices_filtered) if i not in remove_L_indices]

In [ ]:
tapping_R_indices_corrected = remove_long_taps(tapping_R_indices_corrected, fs=512, time_threshold=5)
tapping_L_indices_corrected = remove_long_taps(tapping_L_indices_corrected, fs=512, time_threshold=5)

#### load corrected taps

In [ ]:
with open(DATA_IO.path_events + SUB + "_updated_taps", 'rb') as handle:
    sub_taps = pickle.load(handle)
    
tapping_R_indices_corrected = sub_taps["right"]
tapping_L_indices_corrected = sub_taps["left"]

In [ ]:
taps = get_tapping_count_df(tapping_period_indices, tapping_R_indices_corrected, tapping_L_indices_corrected)
plot_tapping_count_by_tap_periods(taps)

#### plot corrected taps

In [ ]:
plot_aligned_acceleremetor_recordings(tapping_period_indices,  tapping_R_indices_corrected, tapping_L_indices_corrected, 
                                      fs=512,  plot_type = "nan-filled", alignment = "onset")
plot_aligned_acceleremetor_recordings(tapping_period_indices,  tapping_R_indices_corrected, tapping_L_indices_corrected, 
                                      fs=512,  plot_type = "nan-filled", alignment = "offset")
#plot_aligned_acceleremetor_recordings(tapping_period_indices,  tapping_R_indices, tapping_L_indices, 
#                                      fs=512,  plot_type = "raw", alignment = "onset")
#plot_aligned_acceleremetor_recordings(tapping_period_indices,  tapping_R_indices, tapping_L_indices, 
#                                      fs=512,  plot_type = "raw", alignment = "offset")

#### save corrected taps

In [ ]:
sub_taps          = {}
sub_taps["right"] = tapping_R_indices_corrected
sub_taps["left"]  = tapping_L_indices_corrected
with open(DATA_IO.path_events + SUB + "_updated_taps", 'wb') as handle:
    pickle.dump(sub_taps, handle, protocol=pickle.HIGHEST_PROTOCOL)